# Part 1

In [22]:
grid = []

def read_data():
    data = []

    with open('data/day-12.data') as f:
        lines = f.readlines()

        for line in lines:
            data.append([*line.strip()])

    return data

grid = read_data()

In [2]:
class Node:
    def __init__(self, position, label, parent, columns):
        self.position = position
        self.id = position[0]*columns + position[1]
        self.label = label
        self.parent = parent

        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.id == other.id

    def __lt__(self, other):
        return self.f < other.f

    def __gt__(self, other):
        return self.f > other.f

In [3]:
def get_path(current_node):
    path = []
    current = current_node

    while current:
        path.append(current.position)
        current = current.parent

    return path[::-1]  # Return reversed path

In [4]:
def is_accessible(source_label, dest_label):
    if dest_label == 'E': dest_label = 'z'

    return source_label == 'S' or ord(source_label) - ord(dest_label) > -2

In [5]:
def get_manhattan_dist(pos_1, pos_2):
    return abs(pos_1[0] - pos_2[0]) + abs(pos_1[1] - pos_2[1])


def get_euclidean_dist(pos_1, pos_2):
    x_dist = pos_1[0] - pos_2[0]
    y_dist = pos_1[1] - pos_2[1]

    return x_dist**2 + y_dist**2

In [6]:
import heapq
import time



'''
https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2
https://gist.github.com/ryancollingwood/32446307e976a11a1185a5394d6657bc
'''
def a_star():
    start_time = time.time()

    global grid

    columns = len(grid[0])
    rows = len(grid)

    start_node = None

    for i, row in enumerate(grid):
        if start_node: break

        for j, item in enumerate(row):
            if item == 'S':
                start_node = Node((i, j), grid[i][j], None, columns)
                break

    end_node = None

    for i, row in enumerate(grid):
        if end_node: break

        for j, item in enumerate(row):
            if item == 'E':
                end_node = Node((i, j), grid[i][j], None, columns)
                break

    open_list = []
    closed_ids_set = set()

    heapq.heapify(open_list)
    heapq.heappush(open_list, start_node)

    outer_iterations = 0
    max_iterations = 1000000

    while len(open_list) > 0:
        outer_iterations += 1

        if outer_iterations > max_iterations:
            print('Exceeded max iterations!')
            return get_path(current_node)

        current_node = heapq.heappop(open_list)
        closed_ids_set.add(current_node.id)

        if current_node.label == 'E':
            path = get_path(current_node)

            print('Target reached!')
            print('Time elapsed:', time.time() - start_time)
            print(path)

            return path

        adjacent_nodes = []
        position_deltas = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for position_delta in position_deltas:
            adjacent_position = (
                current_node.position[0] + position_delta[0],
                current_node.position[1] + position_delta[1]
            )

            if adjacent_position[0] < 0 or adjacent_position[1] < 0 \
                    or adjacent_position[0] > rows - 1 or adjacent_position[1] > columns - 1:
                continue

            adjacent_label = grid[adjacent_position[0]][adjacent_position[1]]

            if not is_accessible(current_node.label, adjacent_label):
                continue

            new_node = Node(adjacent_position, adjacent_label, current_node, columns)

            adjacent_nodes.append(new_node)

        for adjacent_node in adjacent_nodes:
            if adjacent_node.id in closed_ids_set: continue

            h = get_manhattan_dist(adjacent_node.position, end_node.position)
            g = current_node.g + 1
            adjacent_node.f = g + h
            adjacent_node.g = g

            adj_node_in_open_list = next((n for n in open_list if n == adjacent_node and n.f <= adjacent_node.f), None)

            if adj_node_in_open_list is not None: continue

            heapq.heappush(open_list, adjacent_node)

    print('Time elapsed:', time.time() - start_time)
    print('Path not found.')

path_nodes = a_star()

Target reached!
Time elapsed: 0.0
[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (3, 7), (2, 7), (1, 7), (0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (1, 3), (2, 3), (3, 3), (3, 4), (3, 5), (3, 6), (2, 6), (1, 6), (1, 5), (1, 4), (2, 4), (2, 5)]


In [7]:
len(path_nodes) - 1

31

# Part 2

In [31]:
def is_accessible_2(source_label, dest_label):
    if source_label == 'E': source_label = 'z'
    if dest_label == '~': dest_label = 'a'

    return source_label == 'E' or ord(source_label) - ord(dest_label) < 2

In [44]:
def a_star_2(end_position):
    global grid

    columns = len(grid[0])
    rows = len(grid)

    start_node = None

    for i, row in enumerate(grid):
        if start_node: break

        for j, item in enumerate(row):
            if item == 'E':
                start_node = Node((i, j), grid[i][j], None, columns)
                break

    end_node = Node(end_position, 'X', None, columns)

    open_list = []
    closed_ids_set = set()

    heapq.heapify(open_list)
    heapq.heappush(open_list, start_node)

    outer_iterations = 0
    max_iterations = 1000000

    while open_list:
        outer_iterations += 1

        if outer_iterations > max_iterations:
            print('Exceeded max iterations!')
            return get_path(current_node)

        current_node = heapq.heappop(open_list)
        closed_ids_set.add(current_node.id)

        if current_node.label == '~':
            path = get_path(current_node)

            # print('Target reached!', len(path) - 1)

            return len(path) - 1

        adjacent_nodes = []
        position_deltas = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for position_delta in position_deltas:
            adjacent_position = (
                current_node.position[0] + position_delta[0],
                current_node.position[1] + position_delta[1]
            )

            if adjacent_position[0] < 0 or adjacent_position[1] < 0 \
                    or adjacent_position[0] > rows - 1 or adjacent_position[1] > columns - 1:
                continue

            adjacent_label = grid[adjacent_position[0]][adjacent_position[1]]

            if not is_accessible_2(current_node.label, adjacent_label):
                continue

            new_node = Node(adjacent_position, adjacent_label, current_node, columns)

            adjacent_nodes.append(new_node)

        for adjacent_node in adjacent_nodes:
            if adjacent_node.id in closed_ids_set: continue

            h = get_manhattan_dist(adjacent_node.position, end_node.position)
            g = current_node.g + 1
            adjacent_node.f = g + h
            adjacent_node.g = g

            adj_node_in_open_list = next((n for n in open_list if n == adjacent_node and n.f <= adjacent_node.f), None)

            if adj_node_in_open_list is not None: continue

            heapq.heappush(open_list, adjacent_node)

    # print('Path not found.')
    return 9999999999999

In [45]:
grid = read_data()
path_lengths = []

for i in range(len(grid)):
    for j in range(len(grid[i])):
        if grid[i][j] == 'a':
            grid[i][j] = '~'
            path_lengths.append(a_star_2((i, j)))
            grid[i][j] = '|'

path_lengths

KeyboardInterrupt: 

In [43]:
path_lengths.sort()

path_lengths

[363,
 364,
 364,
 365,
 365,
 366,
 366,
 367,
 367,
 368,
 368,
 369,
 369,
 370,
 371,
 371,
 371,
 372,
 372,
 372,
 373,
 373,
 373,
 373,
 373,
 374,
 374,
 374,
 374,
 374,
 374,
 375,
 375,
 375,
 375,
 375,
 375,
 375,
 376,
 376,
 376,
 376,
 376,
 376,
 376,
 376,
 377,
 377,
 377,
 377,
 377,
 377,
 378,
 378,
 378,
 378,
 379,
 379,
 379,
 380,
 380,
 381,
 381,
 382,
 382,
 382,
 383,
 383,
 383,
 383,
 384,
 384,
 384,
 384,
 384,
 385,
 385,
 385,
 385,
 385,
 385,
 386,
 386,
 386,
 386,
 386,
 386,
 386,
 387,
 387,
 387,
 387,
 387,
 387,
 387,
 388,
 388,
 388,
 388,
 388,
 388,
 388,
 388,
 388,
 389,
 389,
 389,
 389,
 389,
 389,
 389,
 389,
 389,
 389,
 389,
 389,
 389,
 390,
 390,
 390,
 390,
 390,
 390,
 390,
 390,
 390,
 390,
 390,
 391,
 391,
 391,
 391,
 391,
 391,
 391,
 391,
 392,
 392,
 392,
 392,
 392,
 392,
 392,
 392,
 392,
 393,
 393,
 393,
 393,
 393,
 393,
 393,
 393,
 393,
 394,
 394,
 394,
 394,
 394,
 394,
 394,
 394,
 394,
 394,
 395,
 395,
 395